In [2]:
import os
import gym
import gym_donkeycar
import numpy as np
import random
import tensorflow
import shimmy
from rich.live import Live
import stable_baselines3
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import PPO

In [3]:
DONKEY_GYM = True
DONKEY_SIM_PATH = "remote"
DONKEY_GYM_ENV_NAME = "donkey-generated-track-v0"
body_style = "car01"
body_rgb = (128, 128, 128)
car_name = "PPO3"
font_size = 100
WEB_CONTROL_PORT = int(os.getenv("WEB_CONTROL_PORT", 8887))
port = 9091

In [4]:
folder = '/mnt/d/donkeycar_model/PPO3/'

In [5]:
conf = { "DONKEY_GYM":DONKEY_GYM, "DONKEY_SIM_PATH":DONKEY_SIM_PATH, "DONKEY_GYM_ENV_NAME":DONKEY_GYM_ENV_NAME, 
         "body_style":body_style, "body_rgb":body_rgb, "car_name":car_name, "font_size":font_size, 
         "WEB_CONTROL_PORT" : WEB_CONTROL_PORT, "port" : port }

In [10]:
env = gym.make("donkey-minimonaco-track-v0", conf=conf)

INFO:gym_donkeycar.core.client:connecting to localhost:9091 
/home/user/miniconda3/envs/donkey/lib/python3.11/site-packages/gym/spaces/box.py:78: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


starting DonkeyGym env
loading scene mini_monaco


INFO:gym_donkeycar.envs.donkey_sim:on need car config
INFO:gym_donkeycar.envs.donkey_sim:sending car config.
INFO:gym_donkeycar.envs.donkey_sim:sim started!
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 32.33 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 32.52 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 32.53 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 32.76 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 32.2 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 32.69 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 3.77 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 32.49 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 32.45 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 3.52 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 32.37 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 32.51 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 32.24 seconds
INFO:gym_donkeycar.envs.don

In [11]:
for i in range(1000):
    # preparation
    if i == 0:
        model = PPO("CnnPolicy", env, n_steps=500, verbose=0, learning_rate=0.0003*0.5 )
        Log={"TestReward":[]}

    # training
    obs = env.reset()
    model.learn(total_timesteps=2500, progress_bar=0)
    model.save(f'{folder}{(i+1)*2500}.pth')

    # testing
    tem = 0
    for _ in range(20):
        obs = env.reset()
        while True:
            action, _states = model.predict(obs, deterministic=True)
            obs, reward, done, info = env.step(action)
            # print (reward, done)
            env.render()
            tem += reward
            if done: break
    Log["TestReward"].append(tem/20)
    print(f"testing {(i+1)*2500} : {tem/20}")
    np.save(f"{folder}/Log_PPO3-{(i+1)*2500}.npy", Log)

/home/user/miniconda3/envs/donkey/lib/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/home/user/miniconda3/envs/donkey/lib/python3.11/site-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 500`, after every 7 untruncated mini-batches, there will be a truncated mini-batch of size 52
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=500 and n_envs=1)
  warnings.warn(
/home/user/miniconda3/envs/donkey/lib/python3.11/site-packages/stable_baselines3/common/utils.py:573: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writab

testing 2500 : 185.11247068491355
testing 5000 : 334.39704018058103
testing 7500 : 38.22020336138151
testing 10000 : 290.59589525966373
testing 12500 : 339.08916299894423
testing 15000 : 74.85773765797478
testing 17500 : 287.63059019855314
testing 20000 : 268.5555194628911
testing 22500 : 2290.0604888723365
testing 25000 : 307.39624196928094
testing 27500 : 135.1527983269416
testing 30000 : 110.19581203299254
testing 32500 : -1.0000007970221527
testing 35000 : -1.0000007609731285
testing 37500 : 15.426144382458313
testing 40000 : 347.4381743983106


KeyboardInterrupt: 

In [12]:
env.close()

In [14]:
env = gym.make("donkey-minimonaco-track-v0", conf=conf)
num = 42500
test_model = PPO.load(f"{folder}{num}.pth")
obs = env.reset()
tem = 0
while True:
    action, _states = test_model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    # print (reward, done)
    env.render()
    tem += reward
    if done: break
print(tem)
env.close()

INFO:gym_donkeycar.core.client:connecting to localhost:9091 
INFO:gym_donkeycar.envs.donkey_sim:on need car config
INFO:gym_donkeycar.envs.donkey_sim:sending car config.
INFO:gym_donkeycar.envs.donkey_sim:sim started!


starting DonkeyGym env


INFO:gym_donkeycar.envs.donkey_sim:New lap time: 36.59 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 36.08 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 36.36 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 36.9 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 36.11 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 36.48 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 36.58 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 36.32 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 36.2 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 36.06 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 36.51 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 36.55 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 36.46 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 35.88 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 36.42 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 36.19 se

20784.638129738538
